# Next Word Prediction

In [2]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense

# Read the text file
with open('/content/sherlock-holm.es_stories_plain-text_advs.txt', 'r', encoding='utf-8') as file:
    text = file.read()

# Now let’s tokenize the text to create a sequence of words:

In [3]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts([text])
total_words = len(tokenizer.word_index) + 1

# Now let’s create input-output pairs by splitting the text into sequences of tokens and forming n-grams from the sequences:

In [4]:
input_sequences = []
for line in text.split('\n'):
    token_list = tokenizer.texts_to_sequences([line])[0]
    for i in range(1, len(token_list)):
        n_gram_sequence = token_list[:i+1]
        input_sequences.append(n_gram_sequence)

In [5]:
max_sequence_len = max([len(seq) for seq in input_sequences])
input_sequences = np.array(pad_sequences(input_sequences, maxlen=max_sequence_len, padding='pre'))

In [6]:
X = input_sequences[:, :-1]
y = input_sequences[:, -1]

In [7]:
y = np.array(tf.keras.utils.to_categorical(y, num_classes=total_words))

In [8]:
model = Sequential()
model.add(Embedding(total_words, 100, input_length=max_sequence_len-1))
model.add(LSTM(150))
model.add(Dense(total_words, activation='softmax'))
print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 17, 100)           820000    
                                                                 
 lstm (LSTM)                 (None, 150)               150600    
                                                                 
 dense (Dense)               (None, 8200)              1238200   
                                                                 
Total params: 2208800 (8.43 MB)
Trainable params: 2208800 (8.43 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
None


The code above defines the model architecture for the next word prediction model. The ‘Sequential’ model is created, which represents a linear stack of layers. The first layer added to the model is the ‘Embedding’ layer, which is responsible for converting the input sequences into dense vectors of fixed size. It takes three arguments:

‘total_words’, which represents the total number of distinct words in the vocabulary;
‘100’, which denotes the dimensionality of the word embeddings;
and ‘input_length’, which specifies the length of the input sequences.
The next layer added is the ‘LSTM’ layer, a type of recurrent neural network (RNN) layer designed for capturing sequential dependencies in the data. It has 150 units, which means it will learn 150 internal representations or memory cells.

Finally, the ‘Dense’ layer is added, which is a fully connected layer that produces the output predictions. It has ‘total_words’ units and uses the ‘softmax’ activation function to convert the predicted scores into probabilities, indicating the likelihood of each word being the next one in the sequence.

Now let’s compile and train the model:

In [9]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(X, y, epochs=100, verbose=1)

Epoch 1/100
3010/3010 [==============================] - 45s 12ms/step - loss: 6.2324 - accuracy: 0.0770
Epoch 2/100
3010/3010 [==============================] - 24s 8ms/step - loss: 5.5011 - accuracy: 0.1235
Epoch 3/100
3010/3010 [==============================] - 24s 8ms/step - loss: 5.1164 - accuracy: 0.1470
Epoch 4/100
3010/3010 [==============================] - 24s 8ms/step - loss: 4.7874 - accuracy: 0.1668
Epoch 5/100
3010/3010 [==============================] - 22s 7ms/step - loss: 4.4854 - accuracy: 0.1821
Epoch 6/100
3010/3010 [==============================] - 24s 8ms/step - loss: 4.1975 - accuracy: 0.2042
Epoch 7/100
3010/3010 [==============================] - 24s 8ms/step - loss: 3.9246 - accuracy: 0.2298
Epoch 8/100
3010/3010 [==============================] - 23s 8ms/step - loss: 3.6628 - accuracy: 0.2603
Epoch 9/100
3010/3010 [==============================] - 24s 8ms/step - loss: 3.4187 - accuracy: 0.2927
Epoch 10/100
3010/3010 [==============================] - 24s 8

In [10]:
seed_text = "I will leave if they"
next_words = 3

for _ in range(next_words):
    token_list = tokenizer.texts_to_sequences([seed_text])[0]
    token_list = pad_sequences([token_list], maxlen=max_sequence_len-1, padding='pre')
    predicted = np.argmax(model.predict(token_list), axis=-1)
    output_word = ""
    for word, index in tokenizer.word_index.items():
        if index == predicted:
            output_word = word
            break
    seed_text += " " + output_word

print(seed_text)

1/1 [==============================] - 0s 19ms/step
I will leave if they are wrong house


# this is how you can build a Next Word Prediction model using Deep Learning and Python programming language.